## Setup

In [58]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('/content/wise_funnel_events_regional.csv')
df['dt'] = pd.to_datetime(df['dt'])

print(f"Period: {df['dt'].min().date()} to {df['dt'].max().date()}")
print(f"Events: {len(df):,} | Users: {df['user_id'].nunique():,}")

Period: 2024-01-01 to 2024-03-01
Events: 73,440 | Users: 40,223


In [57]:
# Event breakdown
print("Events:")
print(df.groupby('event_name')['user_id'].nunique().sort_values(ascending=False))

# Experience × Event
print("\nExperience × Event:")
print(df.groupby(['experience', 'event_name'])['user_id'].nunique().unstack(fill_value=0))

# Region × Event
print("\nRegion × Event:")
print(df.groupby(['region', 'event_name'])['user_id'].nunique().unstack(fill_value=0))

# Platform × Event
print("\nPlatform × Event:")
print(df.groupby(['platform', 'event_name'])['user_id'].nunique().unstack(fill_value=0))

Events:
event_name
Transfer Created        40223
Transfer Funded         18195
Transfer Transferred    10557
Name: user_id, dtype: int64

Experience × Event:
event_name  Transfer Created  Transfer Funded  Transfer Transferred
experience                                                         
Existing               15808             9622                  6041
New                    25234             8946                  4700

Region × Event:
event_name  Transfer Created  Transfer Funded  Transfer Transferred
region                                                             
LatAm                  15695             5097                  3944
NorthAm                15704             7948                  3565
Other                   9393             5273                  3085

Platform × Event:
event_name  Transfer Created  Transfer Funded  Transfer Transferred
platform                                                           
Android                15955             6757             

In [62]:
df

,event_name,dt,user_id,region,platform,experience,week
0,Transfer Funded,2024-02-15,1513280.0,Other,iOS,Existing,2024-02-12
1,Transfer Funded,2024-02-29,1270052.0,Other,iOS,Existing,2024-02-26
2,Transfer Transferred,2024-02-29,1270052.0,Other,iOS,Existing,2024-02-26
3,Transfer Funded,2024-02-25,1399835.0,Other,iOS,Existing,2024-02-19
4,Transfer Transferred,2024-02-25,1399835.0,Other,iOS,Existing,2024-02-19
...,...,...,...,...,...,...,...
73435,Transfer Created,2024-02-13,1892940.0,NorthAm,Android,Existing,2024-02-12
73436,Transfer Created,2024-02-06,1794325.0,NorthAm,Android,Existing,2024-02-05
73437,Transfer Created,2024-02-20,1194664.0,NorthAm,Android,Existing,2024-02-19
73438,Transfer Created,2024-02-29,1498444.0,NorthAm,Android,Existing,2024-02-26


In [83]:
df['week'] = df['dt'].dt.to_period('W').dt.to_timestamp()

weekly = (
    df[df['event_name'].isin([
        'Transfer Created',
        'Transfer Funded',
        'Transfer Transferred'
    ])]
    .groupby(['week', 'experience', 'event_name'])['user_id']
    .nunique()
    .unstack('event_name')
)

weekly = weekly.rename(columns={
    'Transfer Created': 'created',
    'Transfer Funded': 'funded',
    'Transfer Transferred': 'transferred'
})

weekly['conversion'] = (weekly['transferred'] / weekly['created'] * 100).round(1)
weekly['created_to_funded'] = (weekly['funded'] / weekly['created'] * 100).round(1)
weekly = weekly.reset_index()

weekly_df = weekly[['experience','week','created_to_funded','conversion']]
weekly_df = weekly_df.sort_values(['experience','week'], ascending=True)
print(weekly_df)

event_name experience       week  created_to_funded  conversion
0            Existing 2024-01-01               59.1        34.5
2            Existing 2024-01-08               60.3        35.0
4            Existing 2024-01-15               62.0        37.0
6            Existing 2024-01-22               62.4        37.3
8            Existing 2024-01-29               59.3        37.1
10           Existing 2024-02-05               57.8        36.8
12           Existing 2024-02-12               59.6        37.7
14           Existing 2024-02-19               60.3        39.7
16           Existing 2024-02-26               61.2        40.0
1                 New 2024-01-01               40.6        19.5
3                 New 2024-01-08               39.6        18.3
5                 New 2024-01-15               42.2        20.7
7                 New 2024-01-22               41.4        20.4
9                 New 2024-01-29               33.0        17.5
11                New 2024-02-05        

## 1. Build User-Level Funnel

In [2]:
# Aggregate events to user level
user_funnel = df.groupby('user_id').agg({
    'event_name': lambda x: list(x),
    'experience': 'first',
    'region': 'first',
    'platform': 'first'
}).reset_index()

# Create funnel stage flags
user_funnel['created'] = user_funnel['event_name'].apply(
    lambda x: 'Transfer Created' in x
)
user_funnel['funded'] = user_funnel['event_name'].apply(
    lambda x: 'Transfer Funded' in x
)
user_funnel['transferred'] = user_funnel['event_name'].apply(
    lambda x: 'Transfer Transferred' in x
)

In [3]:
# Overall funnel performance
total_created = user_funnel['created'].sum()
total_funded = user_funnel['funded'].sum()
total_transferred = user_funnel['transferred'].sum()

overall_conversion = (total_transferred / total_created * 100)

print(f"Created:     {total_created:,} (100.0%)")
print(f"Funded:      {total_funded:,} ({total_funded/total_created*100:.1f}%)")
print(f"Transferred: {total_transferred:,} ({overall_conversion:.1f}%)")

Created:     40,223 (100.0%)
Funded:      18,195 (45.2%)
Transferred: 10,557 (26.2%)


## 2. Hypothesis Testing

Testing which dimension has the highest variance in conversion rates.

In [4]:
def analyze_segment(df, segment_col):
    """
    Calculate conversion metrics by segment.
    Returns DataFrame with conversion rates and step-by-step breakdown.
    """
    results = []

    for segment in df[segment_col].unique():
        seg_df = df[df[segment_col] == segment]

        created = seg_df['created'].sum()
        funded = seg_df['funded'].sum()
        transferred = seg_df['transferred'].sum()

        results.append({
            'segment': segment,
            'users': created,
            'pct_users': created / df['created'].sum() * 100,
            'conversion': (transferred / created * 100) if created > 0 else 0,
            'created_to_funded': (funded / created * 100) if created > 0 else 0,
            'funded_to_transferred': (transferred / funded * 100) if funded > 0 else 0
        })

    return pd.DataFrame(results).sort_values('conversion', ascending=False)

### H1: Regional differences?

In [46]:
region_analysis = analyze_segment(user_funnel, 'region')
print(region_analysis)

   segment  users  pct_users  conversion  created_to_funded  \
2    Other   9322  23.175795   33.222484          56.468569   
1    LatAm  15446  38.400915   25.482326          32.914670   
0  NorthAm  15455  38.423290   22.801682          50.773213   

   funded_to_transferred  
2              58.833587  
1              77.419355  
0              44.908882  


In [6]:
region_range = region_analysis['conversion'].max() - region_analysis['conversion'].min()
print(f"Conversion range: {region_range:.1f}pp")

Conversion range: 10.4pp


### H2: Platform differences?

In [45]:
platform_analysis = analyze_segment(user_funnel, 'platform')
print(platform_analysis)

   segment  users  pct_users  conversion  created_to_funded  \
1      iOS  13030  32.394401   31.926324          52.394474   
0      Web  11486  28.555801   24.377503          40.719136   
2  Android  15707  39.049797   22.900618          42.598841   

   funded_to_transferred  
1              60.934525  
0              59.867436  
2              53.758780  


In [8]:
platform_range = platform_analysis['conversion'].max() - platform_analysis['conversion'].min()
print(f"Conversion range: {platform_range:.1f}pp")

Conversion range: 9.0pp


### H3: User experience (New vs Existing)?

In [44]:
experience_analysis = analyze_segment(user_funnel, 'experience')
print(experience_analysis)

    segment  users  pct_users  conversion  created_to_funded  \
1  Existing  15557  38.676876   39.596323          61.586424   
0       New  24666  61.323124   17.826157          34.922565   

   funded_to_transferred  
1              64.293915  
0              51.044811  


In [10]:
exp_range = experience_analysis['conversion'].max() - experience_analysis['conversion'].min()
print(f"Conversion range: {exp_range:.1f}pp")

Conversion range: 21.8pp


### Compare variance across dimensions

In [41]:
variance_comparison = pd.DataFrame({
    'Dimension': ['User Experience', 'Region', 'Platform'],
    'Conv Range (pp)': [exp_range, region_range, platform_range]

}).sort_values('Conv Range (pp)', ascending=False)

print(variance_comparison)

         Dimension  Conv Range (pp)
0  User Experience        21.770166
1           Region        10.420802
2         Platform         9.025706


## 3. Root Cause Analysis

In [26]:
# Step 1: Created → Funded
payment_funnel = user_funnel.groupby('experience').apply(
    lambda x: pd.Series({
        'users': x['created'].sum(),
        'funded': x['funded'].sum(),
        'rate': (x['funded'].sum() / x['created'].sum() * 100)
    })
).reset_index()

print("Payment Step (Created → Funded):")
print(payment_funnel)

payment_gap = payment_funnel[payment_funnel['experience']=='Existing']['rate'].values[0] - \
              payment_funnel[payment_funnel['experience']=='New']['rate'].values[0]
print(f"\nGap: {payment_gap:.1f}pp")


Payment Step (Created → Funded):
  experience    users  funded       rate
0   Existing  15557.0  9581.0  61.586424
1        New  24666.0  8614.0  34.922565

Gap: 26.7pp


In [27]:
# Step 2: Funded → Transferred
payout_funnel = user_funnel[user_funnel['funded']==True].groupby('experience').apply(
    lambda x: pd.Series({
        'funded': x['funded'].sum(),
        'transferred': x['transferred'].sum(),
        'rate': (x['transferred'].sum() / x['funded'].sum() * 100)
    })
).reset_index()

print("Payout Step (Funded → Transferred):")
print(payout_funnel)

payout_gap = payout_funnel[payout_funnel['experience']=='Existing']['rate'].values[0] - \
             payout_funnel[payout_funnel['experience']=='New']['rate'].values[0]
print(f"\nGap: {payout_gap:.1f}pp")

Payout Step (Funded → Transferred):
  experience  funded  transferred       rate
0   Existing  9581.0       6160.0  64.293915
1        New  8614.0       4397.0  51.044811

Gap: 13.2pp


## 4. Validation


In [19]:
# Cross-analyze Region × Experience
cross_analysis = user_funnel.groupby(['region', 'experience']).apply(
    lambda x: pd.Series({
        'users': x['created'].sum(),
        'created_to_funded': (x['funded'].sum() / x['created'].sum() * 100) if x['created'].sum() > 0 else 0,
        'overall_conversion': (x['transferred'].sum() / x['created'].sum() * 100) if x['created'].sum() > 0 else 0
    })
).reset_index()

# Pivot to see pattern
pivot_c2f = cross_analysis.pivot(index='region', columns='experience', values='created_to_funded')
print("Created→Funded by Region × Experience:")
print(pivot_c2f)

Created→Funded by Region × Experience:
experience   Existing        New
region                          
LatAm       50.807137  17.843511
NorthAm     74.186123  45.037860
Other       68.518519  39.477766


In [16]:
# Calculate gaps by region
print("\nGap analysis:")
for region in cross_analysis['region'].unique():
    new_rate = cross_analysis[
        (cross_analysis['region']==region) &
        (cross_analysis['experience']=='New')
    ]['created_to_funded'].values[0]

    existing_rate = cross_analysis[
        (cross_analysis['region']==region) &
        (cross_analysis['experience']=='Existing')
    ]['created_to_funded'].values[0]

    gap = existing_rate - new_rate
    print(f"{region:8s}: New {new_rate:5.1f}% vs Existing {existing_rate:5.1f}% (gap: {gap:5.1f}pp)")


Gap analysis:
LatAm   : New  17.8% vs Existing  50.8% (gap:  33.0pp)
NorthAm : New  45.0% vs Existing  74.2% (gap:  29.1pp)
Other   : New  39.5% vs Existing  68.5% (gap:  29.0pp)


## 5. Business Impact - Multiple Scenarios

In [40]:
# Current funnel breakdown by step
new_created = user_funnel[user_funnel['experience']=='New']['created'].sum()
new_funded = user_funnel[user_funnel['experience']=='New']['funded'].sum()
new_transferred = user_funnel[user_funnel['experience']=='New']['transferred'].sum()

current_c2f_new = (new_funded / new_created * 100)
current_f2t_new = (new_transferred / new_funded * 100)

existing_created = user_funnel[user_funnel['experience']=='Existing']['created'].sum()
existing_funded = user_funnel[user_funnel['experience']=='Existing']['funded'].sum()
existing_c2f = (existing_funded / existing_created * 100)

payment_gap = existing_c2f - current_c2f_new
total_transfers = user_funnel['transferred'].sum()

# Payment step comparison
payment_comparison = pd.DataFrame({
    'Experience': ['New', 'Existing', 'Gap'],
    'Created→Funded': [
        f"{current_c2f_new:.1f}%",
        f"{existing_c2f:.1f}%",
        f"{payment_gap:.1f}pp"
    ]
})

print(payment_comparison)

  Experience Created→Funded
0        New          34.9%
1   Existing          61.6%
2        Gap         26.7pp


In [39]:
# Scenarios: Improve payment step
scenarios = [
    (0.30, 'Conservative'),
    (0.50, 'Base Case'),
    (0.70, 'Optimistic'),
    (1.00, 'Best Case')
]

scenario_results = []

for gap_closure, name in scenarios:  # ← Removido 'action'
    improved_c2f = current_c2f_new + (payment_gap * gap_closure)
    new_overall_conv = (improved_c2f / 100) * (current_f2t_new / 100) * 100

    new_potential = int(new_created * (new_overall_conv / 100))
    additional = new_potential - new_transferred
    pct_increase = (additional / total_transfers * 100)

    scenario_results.append({
        'Scenario': f"{name} ({gap_closure*100:.0f}%)",
        'Payment Rate': f"{improved_c2f:.1f}%",
        'Overall Conv.': f"{new_overall_conv:.1f}%",
        'Additional Transfers': f"+{additional:,}",
        'Impact': f"+{pct_increase:.1f}%"
    })

print(pd.DataFrame(scenario_results))

             Scenario Payment Rate Overall Conv. Additional Transfers  Impact
0  Conservative (30%)        42.9%         21.9%               +1,007   +9.5%
1     Base Case (50%)        48.3%         24.6%               +1,678  +15.9%
2    Optimistic (70%)        53.6%         27.4%               +2,350  +22.3%
3    Best Case (100%)        61.6%         31.4%               +3,357  +31.8%
